### 좋아요수 / 댓글 분산 적용 기반 추천(명곡?)
- Input(title, artist) : contains로 찾기 => 결과 songId 한 개만 반환하도록 한다.
- I am 찾기 -> 전체 (좋아요+댓글) -> 3번째 분산 데이터들에서 -> 가사분석해서 비슷한 노래 찾기 top5
- I am 찾기 -> 가사분석해서 비슷한 노래 찾기 -> 전체 (좋아요+댓글) -> 3번째 분산 데이터들에서 top5
- 반환된 songId 들의 정보 출력

In [28]:
import numpy as np
import pandas as pd
import joblib, re, string

In [41]:
df = pd.read_csv('data/melon_song_v3.csv')
cosine_sim = joblib.load('data/melon_cosine_sim.sim')
df.tail(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq,total,ly_summary
6245,34183979,인생 한잔,정은지,포크/블루스 국내드라마,술꾼도시여자들 OST Part 2,빨간양말,INAN,20211029,https://cdnimg.melon.co.kr/cm2/album/images/10...,12,4512,사랑도 이별도 오늘은 잠시 미뤄보자\n불안한 청춘도 내일로 잠시 미뤄두자\n떠나버린...,506225967 434154554,사랑 이별 오늘 잠시 미루다 보다 불안하다 청춘 내일 잠시 미루다 두다 떠나다 버리...,NaN
6246,31191637,6 o'clock,볼빨간사춘기,인디음악 포크/블루스,Red Diary 'Hidden Track',안지영,안지영,20180717,https://cdnimg.melon.co.kr/cm/album/images/101...,87,15668,It's 6 o'clock\n너와 나는 시간이 빨라서\n시간이 달라서\n어쩔 수 없...,458080033 442957041,It ' s 6 o ' clock 너 와 나 시간 빨 라서 시간 달라 서 어쩔 없이...,NaN


In [43]:
df.ly_summary.isna().sum()

2

In [30]:
# 데이터 처리를 위한 작업
df.date.fillna(0, inplace=True)
df['date'] = df.date.astype(int).astype(str)
df.fillna('', inplace=True)
# df['comment_like_total'] = df.comment + df.like
df['songId'] = df.songId.astype(str)

In [38]:
# 한글 불용어 처리
with open('data/한글불용어.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]
stop_words.extend('은 는 를 도 을 며 의 에 게 니 거 로 요 과 래 랑 파 여 에게'.split())

In [39]:
from konlpy.tag import Okt
okt = Okt()

In [40]:
lyrics = []
for lyric in df.lyric:
    lyric = lyric.replace('\n', ' ')
    morphs = okt.morphs(lyric, stem=True)
    tmp = [word for word in morphs if word not in stop_words]
    lyrics.append(' '.join(tmp))
df['morphs'] = lyrics

In [41]:
# 학습할 데이터열 생성
df['total'] = df.morphs + (' ' + df.title) + (' ' + df.artist) * 2 + (' ' + df.composer) * 2 + (' ' + df.lyricist) * 2 + (' ' + df.genre) * 3

In [42]:
df.set_index('songId', inplace=True)
df.reset_index(inplace=True)

In [4]:

from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
total_tv = tvect.fit_transform(df.total)

In [10]:
# 장르가 힙합이나 댄스 : 결과 안 좋거나 거의 비슷하다. 
# from sklearn.feature_extraction.text import CountVectorizer
# cvect = CountVectorizer(stop_words='english')
# total_cv = cvect.fit_transform(df.total)

In [31]:
indices = pd.Series(df.index, index=df.songId)

In [45]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(total_tv)

In [51]:
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_sim = cosine_similarity(total_cv)

In [32]:
def get_recommendation(songId, cos_sim):
    index = indices[songId]
    sim_scores = pd.Series(cos_sim[index])
    song_indices = sim_scores.sort_values(ascending=False).head(31).tail(30).index
    return df.songId.iloc[song_indices]

In [26]:
df[df.title.str.contains('성시경')]

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq,total
3129,3853977,뜨거운 안녕 (Feat. 성시경),싸이 (PSY),랩/힙합,싸이6甲 Part.1,유희열,싸이 (PSY),20120715,https://cdnimg.melon.co.kr/cm/album/images/021...,107,79115,PSY collaboration\nTOY introducing 성발라\n불타 올랐던...,445029956 429291289,PSY collaboration TOY introducing 성 발라 불타 오르다 ...
4245,34997080,감동이야 (feat. 성시경),싸이 (PSY),랩/힙합,싸다9,싸이 (PSY),화이트99,20220429,https://cdnimg.melon.co.kr/cm2/album/images/10...,122,19021,넌 역시 감동이야 꿈을 꾸는 듯해\n넌 역시 감동이야 품에 품은 듯해 \n영원을 믿...,439324772 418156677,넌 역시 감동 이야 꿈 꾸다 듯 하다 넌 역시 감동 이야 품 품다 듯 하다 영원 믿...


In [33]:

# 키워드 바꿔가면서 찾아보기
key_title = '안녕'
key_artist = '성시경'
print(key_title, key_artist)

key_title = re.sub('['+string.punctuation+']', '', key_title)
key_artist = re.sub('['+string.punctuation+']', '', key_artist)

songIds =  df[df.title.str.replace('['+string.punctuation+']','', regex=True).str.contains(key_title, case=False) 
                & df.artist.str.replace('['+string.punctuation+']','', regex=True).str.contains(key_artist, case=False)][['songId', 'title', 'artist', 'album', 'date', 'img']]

songIds

안녕 성시경


,songId,title,artist,album,date,img
586,9646401,"다정하게, 안녕히",성시경,구르미 그린 달빛 OST Part.5,20160914,https://cdnimg.melon.co.kr/cm/album/images/027...
3729,1884329,안녕 나의 사랑,성시경,"여기, 내 맘속에...",20080612,https://cdnimg.melon.co.kr/cm/album/images/003...


In [34]:
find_songid = songIds.songId.values[0]
a = get_recommendation(find_songid, cosine_sim).to_frame()
a.head()

,songId
348,4446485
414,3079872
2605,8302
385,4864206
412,4300699


In [12]:

# df[df['songId'].isin(a.songId[1:6])].to_dict('records')
df[df['songId'].isin(a.songId[1:6])][['songId', 'title', 'artist', 'genre', 'album', 'lyricist', 'composer']]

,songId,title,artist,genre,album,lyricist,composer
834,30989466,그대니까요,다비치,발라드 국내드라마,라이브 OST Part.3,지훈,로코베리
955,36351521,괜찮아 괜찮아,캐리와 친구들,국내뮤지컬,캐리TV 쇼뮤지컬 [캐리와 슈퍼걸스],NaN,NaN
3006,2544949,못해 (Feat. 美),포맨 (4MEN),발라드,The 3rd Generation (Special Album),민연재,윤민수(바이브)
3555,35920493,그대를 알고,송하예,발라드,그대를 알고,윤사라,김도훈 (RBW)
5528,36181010,밥 한 번 먹자,이찬원,성인가요/트로트,ONE,박종근,오승은


In [13]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total = linear_kernel(total_tv, total_tv)
sim_scores = pd.Series(cosine_sim_total[indices[find_songid]])
sim_scores.sort_values(ascending=False).head(11).tail(10)

3661    0.162954
3006    0.148939
5528    0.136449
834     0.136067
955     0.125582
3555    0.125297
5439    0.121875
6063    0.121769
3044    0.119704
3718    0.119436
dtype: float64

In [30]:
# 찾고 싶은 구간 정하기
numbers = df['comment_like_total']
sorted_numbers = np.sort(numbers)
q1 = np.percentile(sorted_numbers, 25)
q2 = np.percentile(sorted_numbers, 50)
filtered_data = df[(df['comment_like_total'] >= q1) & (df['comment_like_total'] < q2)]
filtered_data = filtered_data[['songId', 'comment_like_total']]
filtered_data.songId.values

array(['35837631', '35866717', '35879546', ..., '3080890', '2981597',
       '34183979'], dtype=object)

In [19]:
# 유사도 top 30 중 원하는 구간에 있는 songId 추출(유사도순)
d = a[a['songId'].isin(filtered_data.songId.values)].head(5)
d

,songId
5377,36263930
4835,36421590
6101,31037943
4776,36282049
477,36425393


In [1]:
# filtered_df = df[df['songId'].isin(a.songId.values[:5])].to_dict('records')
filtered_df = df[df['songId'].isin(d.songId.values[:5])][['songId', 'title', 'artist', 'genre', 'album', 'lyricist', 'composer']]
filtered_df

NameError: name 'df' is not defined